Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [4]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [5]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [6]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
print('ready to go')

ready to go


---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [18]:
# SGD with L2 Regularization
train_subset = 10000
beta = 0.1
graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  # Set the l2 loss as an average of the weights
  l2_loss = beta * tf.nn.l2_loss(weights)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_loss
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Loss at step 0: 325.570343
Training accuracy: 7.8%
Validation accuracy: 10.5%
Loss at step 100: 1.012270
Training accuracy: 81.3%
Validation accuracy: 81.0%
Loss at step 200: 0.999542
Training accuracy: 81.3%
Validation accuracy: 81.0%
Loss at step 300: 0.998729
Training accuracy: 81.4%
Validation accuracy: 81.1%
Loss at step 400: 0.998499
Training accuracy: 81.4%
Validation accuracy: 81.1%
Loss at step 500: 0.998427
Training accuracy: 81.4%
Validation accuracy: 81.2%
Loss at step 600: 0.998403
Training accuracy: 81.4%
Validation accuracy: 81.2%
Loss at step 700: 0.998396
Training accuracy: 81.4%
Validation accuracy: 81.2%
Loss at step 800: 0.998393
Training accuracy: 81.4%
Validation accuracy: 81.2%
Test accuracy: 87.6%


In [25]:
# NN with L2 Regularization
batch_size = 128
num_steps = 3001
beta = 0.1

graph_nn = tf.Graph()
with graph_nn.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
 
  # Parametes
  n_input = image_size * image_size
  n_hidden_1 = 150 # 1st layer nodes(features)

  # Store layers weight & bias
  layer_weights = {
    'l1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=0.1)),
    'logits': tf.Variable(tf.random_normal([n_hidden_1, num_labels], stddev=0.1))
  }
  layer_biases = {
    'l1': tf.Variable(tf.random_normal([n_hidden_1], stddev=0.1)),
    'logits': tf.Variable(tf.random_normal([num_labels], stddev=0.1))
  }

  # NN computation(1st layer)
  layer_1 = tf.add(tf.matmul(tf_train_dataset, layer_weights['l1']), layer_biases['l1'])
  layer_1_relu = tf.nn.relu(layer_1)

  # NN Output(Logits)
  logits = tf.add(tf.matmul(layer_1_relu, layer_weights['logits']), layer_biases['logits'])
    
  # Set the l2 loss as an average of the weights
  l2_loss = beta * tf.nn.l2_loss(layer_weights['logits'])

  # Compute softmax and cross entropy of logits  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_loss
  
  # Optimizer: SGD
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  # TODO what do I do here to get other predictions?
  def forward_prop(dataset):
    
    # apply first layer with relu
     prop_layer_1 = tf.add(tf.matmul(dataset, layer_weights['l1']), layer_biases['l1'])
     prop_layer_1_relu = tf.nn.relu(prop_layer_1)

     # apply output layer
     return tf.add(tf.matmul(prop_layer_1_relu, layer_weights['logits']), layer_biases['logits'])

  # now, forward prop the datasets, and set softmax values
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

    
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph_nn) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, 
       loss, 
       train_prediction], 
        feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("SGD + RELU accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 3.576902
Minibatch accuracy: 7.0%
Validation accuracy: 9.4%
Minibatch loss at step 500: 0.771668
Minibatch accuracy: 81.2%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 0.724553
Minibatch accuracy: 82.0%
Validation accuracy: 82.0%
Minibatch loss at step 1500: 0.631046
Minibatch accuracy: 85.2%
Validation accuracy: 82.4%
Minibatch loss at step 2000: 0.775489
Minibatch accuracy: 79.7%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 0.862135
Minibatch accuracy: 77.3%
Validation accuracy: 83.1%
Minibatch loss at step 3000: 0.831017
Minibatch accuracy: 78.1%
Validation accuracy: 83.2%
SGD + RELU accuracy: 89.2%


Initialized
Minibatch loss at step 0: 17.368444
Minibatch accuracy: 10.9%
Validation accuracy: 30.1%
Minibatch loss at step 500: 0.792217
Minibatch accuracy: 82.0%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 0.795392
Minibatch accuracy: 78.1%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.699853
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 0.887771
Minibatch accuracy: 78.9%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 1.017639
Minibatch accuracy: 75.8%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 0.940220
Minibatch accuracy: 75.8%
Validation accuracy: 81.9%
SGD + RELU accuracy: 88.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [26]:
## NN with L2 Regularization

batch_size = 128
num_steps = 3
beta = 0.3

graph_nn = tf.Graph()
with graph_nn.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
 
  # Parametes
  n_input = image_size * image_size
  n_hidden_1 = 200 # 1st layer nodes(features)

  # Store layers weight & bias
  layer_weights = {
    'l1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1], stddev=0.1)),
    'logits': tf.Variable(tf.truncated_normal([n_hidden_1, num_labels], stddev=0.1))
  }
  layer_biases = {
    'l1': tf.Variable(tf.random_normal([n_hidden_1], stddev=0.1)),
    'logits': tf.Variable(tf.random_normal([num_labels], stddev=0.1))
  }

  # NN computation(1st layer)
  layer_1 = tf.add(tf.matmul(tf_train_dataset, layer_weights['l1']), layer_biases['l1'])
  layer_1_relu = tf.nn.relu(layer_1)
    
 
  # NN Output(Logits)
  logits = tf.add(tf.matmul(layer_1_relu, layer_weights['logits']), layer_biases['logits'])
    
  # Set the l2 loss as an average of the weights
  l2_loss = beta * tf.nn.l2_loss(layer_weights['logits'])

  # Compute softmax and cross entropy of logits  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_loss
  
  # Optimizer: SGD
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  # TODO what do I do here to get other predictions?
  def forward_prop(dataset):
    
    # apply first layer with relu
     prop_layer_1 = tf.add(tf.matmul(dataset, layer_weights['l1']), layer_biases['l1'])
     prop_layer_1_relu = tf.nn.relu(prop_layer_1)

     # apply output layer
     return tf.add(tf.matmul(prop_layer_1_relu, layer_weights['logits']), layer_biases['logits'])

  # now, forward prop the datasets, and set softmax values
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

    
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph_nn) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, 
       loss, 
       train_prediction], 
        feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("SGD + RELU accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

  # TODO this gives out 88.2% accurracy, which is less than the 89.5% in A2, optimize this

Initialized
Minibatch loss at step 0: 5.209593
Minibatch accuracy: 11.7%
Validation accuracy: 10.9%
SGD + RELU accuracy: 19.6%


In [ ]:
The accuracy drops to 19.6%, much lower than above. 

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [33]:
## NN with L2 Regularization and Dropout

batch_size = 128
num_steps = 3001
beta = 0.1

graph_nn = tf.Graph()
with graph_nn.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  keep_prob = tf.placeholder("float")
 
  # Parametes
  n_input = image_size * image_size
  n_hidden_1 = 200 # 1st layer nodes(features)

  # Store layers weight & bias
  layer_weights = {
    'l1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1], stddev=0.3)),
    'logits': tf.Variable(tf.truncated_normal([n_hidden_1, num_labels], stddev=0.3))
  }
  layer_biases = {
    'l1': tf.Variable(tf.random_normal([n_hidden_1], stddev=0.1)),
    'logits': tf.Variable(tf.random_normal([num_labels], stddev=0.1))
  }

  # NN computation(1st layer)
  layer_1 = tf.add(tf.matmul(tf_train_dataset, layer_weights['l1']), layer_biases['l1'])
  layer_1_relu = tf.nn.relu(layer_1)
    
  # Add Dropout to relu output
  layer_1_dropout = tf.nn.dropout(layer_1_relu, keep_prob)
    
  # NN Output(Logits)
  logits = tf.add(tf.matmul(layer_1_dropout, layer_weights['logits']), layer_biases['logits'])
    
  # Set the l2 loss as an average of the weights
  l2_loss = beta * tf.nn.l2_loss(layer_weights['logits'])

  # Compute softmax and cross entropy of logits  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + l2_loss
  
  # Optimizer: SGD
  optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  # TODO what do I do here to get other predictions?
  def forward_prop(dataset):
    
    # apply first layer with relu
     prop_layer_1 = tf.add(tf.matmul(dataset, layer_weights['l1']), layer_biases['l1'])
     prop_layer_1_relu = tf.nn.relu(prop_layer_1)

     # apply output layer
     return tf.add(tf.matmul(prop_layer_1_relu, layer_weights['logits']), layer_biases['logits'])

  # now, forward prop the datasets, and set softmax values
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

    
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph_nn) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.3}
    _, l, predictions = session.run(
      [optimizer, 
       loss, 
       train_prediction], 
        feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("SGD + RELU + Dropout accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))



Initialized
Minibatch loss at step 0: 29.946146
Minibatch accuracy: 8.6%
Validation accuracy: 33.7%
Minibatch loss at step 500: 1.150007
Minibatch accuracy: 77.3%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 1.126354
Minibatch accuracy: 71.1%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 1.009677
Minibatch accuracy: 76.6%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 1.263073
Minibatch accuracy: 68.8%
Validation accuracy: 81.1%
Minibatch loss at step 2500: 1.250964
Minibatch accuracy: 66.4%
Validation accuracy: 77.3%
Minibatch loss at step 3000: 1.119730
Minibatch accuracy: 70.3%
Validation accuracy: 82.6%
SGD + RELU + Dropout accuracy: 89.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
# TODO, GET YOUR DESKTOP RUNNING WITH ITS GPU MIGHT